In [16]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tensorflow_decision_forests

Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f'Found TF-DF {tfdf.__version__}')

Found TF-DF 1.12.0


In [19]:
train_df = pd.read_csv("train.csv")
serving_df = pd.read_csv("test.csv")

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [20]:
#前処理
def preprocess(df):
    df = df.copy()
    #空白で一個ずつのワードに絞って、その中のワードに対してstripで指定した文字を除く文字列を作る
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    #文字列を空白で区切って最後のワードだけ取り出す
    def ticket_number(x):
        return x.split(" ")[-1]
    #文字列の最後以外のワードを_を間に挟んで取り出す
    def ticket_item(x):
        items = x.split(" ")
        if len(items)==1:
            return "None"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)

    return df
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,None
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,None
5,6,0,3,Moran Mr James,male,NaN,0,0,330877,8.4583,NaN,Q,330877,None
6,7,0,1,McCarthy Mr Timothy J,male,54.0,0,0,17463,51.8625,E46,S,17463,None
7,8,0,3,Palsson Master Gosta Leonard,male,2.0,3,1,349909,21.0750,NaN,S,349909,None
8,9,1,3,Johnson Mrs Oscar W Elisabeth Vilhelmina Berg,female,27.0,0,2,347742,11.1333,NaN,S,347742,None
9,10,1,2,Nasser Mrs Nicholas Adele Achem,female,14.0,1,0,237736,30.0708,NaN,C,237736,None


In [21]:
#特徴量の抽出
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("Survived")
input_features.remove("PassengerId")

print(f'Input features:{input_features}')

Input features:['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [22]:
def tokenize_names(features,labels=None):
    #文字列を"Mr. John Doe" → ["Mr.", "John", "Doe"]この感じで分割
    features["Name"] = tf.strings.split(features["Name"])
    return features,labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

In [23]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,#ログの出力を抑える
    #使用する特徴量を明示
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    #featuresにない列は無視
    exclude_non_specified_features=True,
    random_seed=1234)
model.fit(train_ds)

W0000 00:00:1747061041.986911 5261994 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061041.987633 5261994 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061041.987639 5261994 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1747061042.206970 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061042.207116 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061042.207121 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

In [24]:
preds = model.predict(serving_ds)
preds.shape

1/1 [==============================] - 0s 50ms/step


(418, 1)

In [25]:
#モデルの自己評価
self_evaluation = model.make_inspector().evaluation()
print(f'Accuracy:{self_evaluation.accuracy} Loss:{self_evaluation.loss}')

Accuracy:0.804347813129425 Loss:0.8922085165977478


In [26]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,#ログ出力を抑制
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,#指定していない列は無視
    min_examples=1,#各ノードに最低１サンプルあれば分割可
    categorical_algorithm="RANDOM",#カテゴリ変数の分割方法
    #max_depth=4,
    shrinkage=0.05,#学習率
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",#分割基準（多次元に跨る斜めの境界）
    sparse_oblique_normalization="MIN_MAX",#特徴の正規化方法
    sparse_oblique_num_projections_exponent=2.0,#木の分岐に使う射影の数をデータの次元数に応じて増やす
    num_trees=2000,#決定木の数
    #validation_ratio=0.0,
    random_seed=1234,
)
model.fit(train_ds)
self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

W0000 00:00:1747061042.524838 5261994 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061042.524850 5261994 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061042.524852 5261994 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1747061042.641147 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061042.641156 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061042.641160 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

Accuracy: 0.77173912525177 Loss:1.0136744976043701


I0000 00:00:1747061042.873297 5359818 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 1.01367
I0000 00:00:1747061042.873308 5359818 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 83
I0000 00:00:1747061042.876392 5359818 gradient_boosted_trees.cc:279] Truncates the model to 54 tree(s) i.e. 54  iteration(s).
I0000 00:00:1747061042.876848 5359818 gradient_boosted_trees.cc:341] Final model num-trees:54 valid-loss:1.013674 valid-accuracy:0.771739
I0000 00:00:1747061042.877668 5359818 kernel.cc:926] Export model in log directory: /var/folders/xx/9mgb_75s5t99z6yk59t3c9d00000gn/T/tmpn2mav23n with prefix 1a2b18630e5b44f4
I0000 00:00:1747061042.878959 5359818 kernel.cc:944] Save model in resources
I0000 00:00:1747061042.879652 5261994 abstract_model.cc:921] Model self evaluation:
Task: CLASSIFICATION
Label: __LABEL
Loss (BINOMIAL_LOG_LIKELIHOOD): 1.01367

Accuracy: 0.771739  CI95[W][0 1]
Err

In [27]:
model.summary()

Model: "gradient_boosted_trees_model_104"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.431313 ################
    2.        "Pclass"  0.401411 ##############
    3.           "Age"  0.373986 ############
    4.          "Fare"  0.361093 ###########
    5.         "SibSp"  0.292653 #######
    6.         "Parch"  0.255448 ####
    7.          "Name"  0.225272 ##
    8.   "Ticket_item"  0.182111 
    9.      "Embarked"  0.181387 
   10. "Ticket_number"  0.180897 

Variable Importance: NU

In [28]:
#提出用関数
def prediction_to_kaggle_format(model,threshold=0.5):
    proba_survive = model.predict(serving_ds,verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId":serving_df["PassengerId"],
        "Survived":(proba_survive>threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path = "/Users/iwasakitomoya/kaggle/titanic/submission.csv"
    kaggle_predictions.to_csv(path,index=False)
    print(f'Submission exported to {path}')

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)


Submission exported to /Users/iwasakitomoya/kaggle/titanic/submission.csv


In [29]:
#ハイパーパラメータチューニング自動化
tuner = tfdf.tuner.RandomSearch(num_trials=100)#100通りのランダムパラメータ
tuner.choice("min_examples",[2,5,7,10])#ノード分割に必要な最小サンプル数
tuner.choice("categorical_algorithm",["CART","RANDOM"])#カテゴリ変数の扱い方(cart=標準、random=ランダム分割)

local_search_space = tuner.choice("growing_strategy",["LOCAL"])#これがlocalの時のみmax_depth調整
local_search_space.choice("max_depth",[3,4,5,6,8])#決定木の深さ

global_search_space = tuner.choice("growing_strategy",["BEST_FIRST_GLOBAL"],merge=True)#グローバル戦力の場合はノード数の上限で木の大きさを制限
global_search_space.choice("max_num_nodes",[16,32,64,128,256])

tuner.choice("shrinkage",[0.02,0.05,0.10,0.15])#学習率、小さいほど汎化しやすい
tuner.choice("num_candidate_attributes_ratio",[0.2,0.5,0.9,1.0])#スプリット(分割)時に考慮する特徴量の割合

tuner.choice("split_axis",["AXIS_ALIGNED"])#分割方法（軸に沿うor斜め分割）
oblique_space = tuner.choice("split_axis",["SPARSE_OBLIQUE"],merge=True)
oblique_space.choice("sparse_oblique_normalization",["NONE","STANDARD_DEVIATION","MIN_MAX"])
oblique_space.choice("sparse_oblique_weights",["BINARY","CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent",[1.0,1.5])
#斜め分割に使うパラメータ群

tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds,verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f'Accuracy:{tuned_self_evaluation.accuracy}Loss:{tuned_self_evaluation.loss}')

Use /var/folders/xx/9mgb_75s5t99z6yk59t3c9d00000gn/T/tmpirfkkzn_ as temporary training directory


W0000 00:00:1747061043.054077 5261994 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061043.054087 5261994 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061043.054091 5261994 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1747061043.170031 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061043.170043 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061043.170047 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false


Accuracy:0.8767123222351074Loss:0.694084644317627


In [31]:
#ensembleで精度向上
predictions = None #全モデルの予測を加算していくための変数
num__predictions = 0#何書い予測を重ねたかカウント

for i in range(100):
    print(f'i:{i}')
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,
        random_seed=i,
        honest=True#各ツリーにバリデーションデータを合わせて過学習を抑制
    )
    model.fit(train_ds,verbose=0)

    sub_predictions = model.predict(serving_ds,verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num__predictions += 1

predictions /= num__predictions

kaggle_predictions = pd.DataFrame({
    "PassengerId":serving_df["PassengerId"],
    "Survived":(predictions >= 0.5).astype(int)
})

make_submission(kaggle_predictions)

i:0


W0000 00:00:1747061077.847009 5261994 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061077.847031 5261994 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1747061077.847041 5261994 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1747061077.973977 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061077.973988 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061077.973992 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

i:1


I0000 00:00:1747061078.336851 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061078.336860 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061078.336864 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^SibSp$"
}
column_guides {
  column_name_pattern: "^Parch$"
}
column_guides {
  column_name_pattern: "^Fare$"
}
column_guides {
  column_name_pattern: "^Cabin$"
}
column_guides {
  column_name_pattern: "^Embarked$"
}
column_guides {
  column_name_pattern: "^Ticket_number$"
}
column_guides {
  column_name_pattern: "^Ticket_item$"
}
default_column_guide {
  categorial {
    max_vocab_

i:2


...

I0000 00:00:1747061116.720780 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061116.720789 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061116.720793 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^SibSp$"
}
column_guides {
  column_name_pattern: "^Parch$"
}
column_guides {
  column_name_pattern: "^Fare$"
}
column_guides {
  column_name_pattern: "^Cabin$"
}
column_guides {
  column_name_pattern: "^Embarked$"
}
column_guides {
  column_name_pattern: "^Ticket_number$"
}
column_guides {
  column_name_pattern: "^Ticket_item$"
}
default_column_guide {
  categorial {
    max_vocab_

i:99


I0000 00:00:1747061117.272275 5261994 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1747061117.272284 5261994 kernel.cc:783] Collect training examples
I0000 00:00:1747061117.272288 5261994 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^SibSp$"
}
column_guides {
  column_name_pattern: "^Parch$"
}
column_guides {
  column_name_pattern: "^Fare$"
}
column_guides {
  column_name_pattern: "^Cabin$"
}
column_guides {
  column_name_pattern: "^Embarked$"
}
column_guides {
  column_name_pattern: "^Ticket_number$"
}
column_guides {
  column_name_pattern: "^Ticket_item$"
}
default_column_guide {
  categorial {
    max_vocab_

Submission exported to /Users/iwasakitomoya/kaggle/titanic/submission.csv


I0000 00:00:1747061117.616969 5441095 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.751876
I0000 00:00:1747061117.616983 5441095 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 260
I0000 00:00:1747061117.620325 5441095 gradient_boosted_trees.cc:279] Truncates the model to 231 tree(s) i.e. 231  iteration(s).
I0000 00:00:1747061117.620553 5441095 gradient_boosted_trees.cc:341] Final model num-trees:231 valid-loss:0.751876 valid-accuracy:0.844444
I0000 00:00:1747061117.622047 5441095 kernel.cc:926] Export model in log directory: /var/folders/xx/9mgb_75s5t99z6yk59t3c9d00000gn/T/tmp7c7_wv1c with prefix 016a283c983a4892
I0000 00:00:1747061117.624801 5441095 kernel.cc:944] Save model in resources
I0000 00:00:1747061117.625556 5261994 abstract_model.cc:921] Model self evaluation:
Task: CLASSIFICATION
Label: __LABEL
Loss (BINOMIAL_LOG_LIKELIHOOD): 0.751876

Accuracy: 0.844444  CI95[W][0 